### Pulling genes for RNAseq overlay


In [34]:
import os

import pandas as pd
import polars as pl
import gtfparse
from biomart import BiomartServer


In [3]:
"""
Read and filter hg38 rna annotation file for just genes
"""
hg38_rna_anno = gtfparse.read_gtf("/Users/jkirkland/2023_chavez_rotation/anno/gencode.v38.annotation.gtf")


INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'hgnc_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'ont', 'protein_id', 'ccdsid']


In [4]:
hg38_rna_anno_df = hg38_rna_anno.to_pandas()
genes_hg38_anno = hg38_rna_anno_df[hg38_rna_anno_df["feature"] == "gene"]

In [35]:
"""
Read and setup normalized RNAseq data
"""
medullo_rnaseq_norm = pd.read_csv("/Users/jkirkland/2023_chavez_rotation/data/RNAseq/dkfz_RNAseq_v2_rsem_genes_counts.genesymbol.nodup.renamed.norm.txt", sep="\t")
medullo_rnaseq_norm = medullo_rnaseq_norm.rename(columns={"Unnamed: 0": "gene"})

In [32]:
"""
Pull ensembl gene id's
"""

def get_ensembl_mappings():                                   
    # Set up connection to server                                             
    server = biomart.BiomartServer('http://ensembl.org/biomart')         
    mart = server.datasets['hsapiens_gene_ensembl']                            
                                                                                
    # List the types of data we want                                            
    attributes = ["ensembl_gene_id_version", "hgnc_symbol"]
                                                                                
    # Get the mapping between the attributes                                    
    response = mart.search( {'attributes': attributes})                          

    # Initialize an empty list to collect the rows
    data = []

    for line in response.iter_lines():
        # Decode the line from the response
        line = line.decode('utf-8')
        
        # Split the line by tabs
        line_data = line.split("\t")
        
        # Append the line data to the data list
        data.append(line_data)

    # Create the DataFrame from the list of rows
    columns = attributes  # Replace with actual column names
    df = pd.DataFrame(data, columns=["ensembl_gene_id", "gene_name"])
                                   
                                                                                
              
                                                                                
    return df

ensmbl_ids = get_ensembl_mappings()

In [36]:
medullo_w_gene_id = medullo_rnaseq_norm.merge(ensmbl_ids, how="left", left_on="gene", right_on="gene_name", indicator=True)
medullo_w_gene_id[medullo_w_gene_id["_merge"] == "left_only"]

,gene,MB095,MB106,MB170,MB226,MB247,MB248,MB260,MB164,MB166,...,MB275,MB284,MB088,MB136,MB206,MB266,MB287,ensembl_gene_id,gene_name,_merge
90,FAM214B,838.518196,891.525906,682.195669,396.821920,1148.576998,581.504101,598.159827,2159.201459,1775.038223,...,1040.608555,1819.854765,1096.784422,432.937263,1128.355300,1187.765885,1829.959804,NaN,NaN,left_only
181,KIAA0100,14955.553998,29845.967590,15458.431266,33821.195205,12667.768226,11081.142145,20742.071654,14263.215843,16573.626979,...,26319.935481,22254.556907,11716.461976,26829.720998,14305.965328,20403.661928,20716.165164,NaN,NaN,left_only
321,TMEM159,1190.078233,65.856930,2616.688909,46.190911,1430.429549,110.485779,128.177106,604.515500,1940.127343,...,1954.773354,2778.509342,910.224125,111.503715,1315.965688,1889.942220,1171.620606,NaN,NaN,left_only
367,ACPP,3.563108,11.795271,66.555675,10.497934,12.254459,11.630082,28.483801,0.000000,6.603565,...,7.853649,7.415008,19.012514,15.830774,4.488287,26.702480,3.985104,NaN,NaN,left_only
399,ZNF806,15.440137,0.982939,57.798349,6.298761,2.228083,20.934148,17.592936,28.931472,9.244991,...,9.424379,33.897178,28.518772,0.000000,1.795315,36.592288,9.564250,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48573,AC096992.2,213.786509,76.669262,126.105490,171.116330,153.737755,96.529681,127.339347,130.952980,81.884204,...,95.029159,169.485892,95.062572,38.544494,130.160317,353.066129,211.210517,NaN,NaN,left_only
48574,AC104836.1,29.692571,40.300510,40.283698,10.497934,43.447626,19.771139,14.241901,15.227091,58.111370,...,2.356095,9.533581,2.376564,4.129767,24.236749,114.721767,23.910625,NaN,NaN,left_only
48575,AC008264.2,326.618277,70.771626,116.472431,86.083062,230.606633,32.564230,103.882099,16.749800,46.224954,...,45.551167,32.837892,21.389079,33.726432,36.803952,135.490363,17.534458,NaN,NaN,left_only
48576,AP000229.1,9.501623,14.744089,3.502930,1.049793,5.570209,0.000000,3.351035,79.180872,87.167056,...,91.102334,776.457243,425.405009,268.434869,485.632631,337.242437,441.549535,NaN,NaN,left_only


In [23]:
"""
Subsetting Datframe for needed col_names
"""

col_names = [
    "gene", "MB095", "MB106", "MB170", "MB226", "MB247", "MB248", "MB260", "MB164", "MB166",
    "MB271", "MB277", "MB278", "MB288", "MB091", "MB099", "MB118", "MB174", "MB177",
    "MB199", "MB227", "MB264", "MB265", "MB269", "MB270", "MB281", "MB102", "MB104",
    "MB234", "MB239", "MB244", "MB268", "MB274", "MB275", "MB284", "MB088", "MB136",
    "MB206", "MB266", "MB287", "seqname", "source", "feature", "start", "end", "strand",
    "gene_id", "gene_type", "level", "hgnc_id", "havana_gene"
]
merged_subset = medullo_rnaseq_norm.merge(genes_hg38_anno, how="left", left_on="gene", right_on="gene_name", indicator=True)
unmerged = merged_subset[~merged_subset['_merge'].isin(["both"])]

merged_subset.head()
# merge_filtered = merged_subset[merged_subset.columns.intersection(col_names)]


,gene,MB095,MB106,MB170,MB226,MB247,MB248,MB260,MB164,MB166,...,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid,_merge
0,TSPAN6,1604.586519,1360.387931,3453.889243,1668.121774,2287.127620,2326.016403,1297.688477,1659.752884,1499.009214,...,,,,,,,,,,both
1,TNMD,5.938514,0.000000,6.130128,0.000000,0.000000,2.326016,0.000000,0.000000,1.320713,...,,,,,,,,,,both
2,DPM1,1340.916492,2214.562145,1980.031333,2363.085030,2320.548871,3843.742106,2850.893406,2168.337713,2234.646334,...,,,,,,,,,,both
3,SCYL3,1690.101123,2397.388847,1733.950482,2181.470765,3040.219812,2956.366849,1838.880701,1516.618232,3020.470548,...,,,,,,,,,,both
4,C1orf112,972.728615,2371.832426,3026.531750,3206.069161,1551.860095,2465.577387,1409.110405,922.761695,1419.766436,...,,,,,,,,,,both


In [22]:
unmerged

,gene,MB095,MB106,MB170,MB226,MB247,MB248,MB260,MB164,MB166,...,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid,_merge
248,METTL13,2026.221023,5569.333828,2400.382965,4618.041336,3654.056792,4307.782379,5146.352687,2352.585510,3555.359298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
350,ACPP,3.563108,11.795271,66.555675,10.497934,12.254459,11.630082,28.483801,0.000000,6.603565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
384,CXorf56,2053.538188,1545.180511,2706.013631,2845.990011,1779.124603,4592.719388,2217.547707,4897.032364,3190.842520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
424,LOC100128242,37186.975515,30982.245370,21989.644748,32094.284999,12458.328385,12543.043455,11029.095413,19667.310326,19337.879212,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
492,SARS,8674.981449,14429.548288,10547.323032,10981.889159,10570.027695,21017.884220,10325.377969,13197.319495,16668.718313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44435,AC096992.2,213.786509,76.669262,126.105490,171.116330,153.737755,96.529681,127.339347,130.952980,81.884204,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
44436,AC104836.1,29.692571,40.300510,40.283698,10.497934,43.447626,19.771139,14.241901,15.227091,58.111370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
44437,AC008264.2,326.618277,70.771626,116.472431,86.083062,230.606633,32.564230,103.882099,16.749800,46.224954,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
44438,AP000229.1,9.501623,14.744089,3.502930,1.049793,5.570209,0.000000,3.351035,79.180872,87.167056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [37]:
"""
Export to CSV
"""
merge_filtered.to_csv("medullo_rnaseq_annotated.csv", index=False)